### Парсинг данных и создание датафрейма

In [19]:
import os
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

# %matplotlib notebook

In [20]:
runN = "*" # If take all runs set runN = "*"
paths = glob.glob(f"build/Run{runN}_nt_ParticleData_t*.csv")

In [21]:
with open(paths[0], 'r') as f:
    print(f"Headers from file: {paths[0]}")
    
#     lines = f.readline().strip()
#     print(lines)
#     columns = lines[1:]
#     columns = columns.split(",")
#     meta_len = 1

    hlines = []
    while True:
        line = f.readline()
        if len(line) == 0 or line[0] != '#':
            break
        hlines.append(line)
    title = hlines[1].replace('#title ', '')[:-1]
    sep = chr(int(hlines[2].replace('#separator ', '')[:-1]))
    vsc_sep = chr(int(hlines[3].replace('#vector_separator ', '')[:-1]))
    columns = [' '.join(l.split(' ')[2:])[:-1] for l in hlines[4:]]
    meta_len = len(hlines)
    print(f'Metarows: {meta_len}\nColumns: {columns}')

Headers from file: build/Run0_nt_ParticleData_t3.csv
Metarows: 8
Columns: ['particle_name', 'creator_process', 'parentID', 'energy']


In [22]:
data = pd.DataFrame()
for path in paths:
    item = pd.read_csv(path, skiprows=meta_len, names=columns)
    data = data.append(item, ignore_index=True)

/tmp/ipykernel_17142/1609426551.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(item, ignore_index=True)
/tmp/ipykernel_17142/1609426551.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(item, ignore_index=True)
/tmp/ipykernel_17142/1609426551.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(item, ignore_index=True)


e - elecron, Brem - тормозное излучение, Ionic - ионизационные потери (неупругое)

conv - обратная аннигилияция (фотоны в электроны и позитроны)

annihil - аннигилиция

phot - фотоэффект

In [18]:
set(data['creator_process'])

{'annihil', 'compt', 'conv', 'eBrem', 'eIoni', 'phot'}

In [26]:
filt = data[data['creator_process'] == 'conv']
filt = filt[filt['particle_name'] == 'e+' | filt['particle_name'] == 'e-']

TypeError: Cannot perform 'ror_' with a dtyped [object] array and scalar of type [bool]

In [15]:
len(data[data["parentID"] == 1])

59

In [16]:
secondary = data[data["parentID"] == 1]
secondary_e_energy = sum(secondary[secondary['particle_name']=='e-']['energy'])

In [17]:
primary = data[data["parentID"] == 0]
primary_energy = sum(primary['energy'])
secondary_e_energy/primary_energy

ZeroDivisionError: float division by zero

In [ ]:
plt.hist(secondary['energy']/150000, bins=100)
plt.show()